<a href="https://colab.research.google.com/github/AlexEMG/DeepLabCut/blob/master/examples/Colab_DEMO_mouse_openfield.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut Toolbox - Colab Demo on topview mouse data
https://github.com/AlexEMG/DeepLabCut

**new: no need to download from GitHub and link to Google Drive! Now you can directly use our demo data!**

Nath\*, Mathis\* et al. *Using DeepLabCut for markerless3D  pose estimation during behavior across species. Nature Protocols, 2019

This notebook demonstrates the necessary steps to use DeepLabCut on our demo data. We provide labeled mouse data from Mathis et al, 2018 Nature Neuroscience.

This demo notebook mostly shows the most simple code to train and evaluate your model, but many of the functions have additional features, so please check out the overview & the protocol paper!

This notebook illustrates how to use the cloud to:

- load demo data
- create a training set
- train a network
- evaluate a network
- analyze a novel video



## [optional] First,  let's look at info about the Colab Environment (and decide, GPU or TPU):

First, go to "Runtime" ->"change runtime type"->select "Python3", and then decide, "GPU or TPU!"" (GPUs are faster!)


In [0]:
!nvcc --version

If you want to use a GPU:

In [0]:
import tensorflow as tf
tf.__version__

In [0]:
#let's make sure we see a GPU:
tf.test.gpu_device_name()

#from tensorflow.python.client import device_lib
#device_lib.list_local_devices()

If you selected to use a TPU: 

In [0]:
#let's make sure we see a TPU:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)

## [Mandatory starting point!] Using DeepLabCut: 

In [0]:
# Clone the entire deeplabcut repo so we can use the demo data:
!git clone -l -s git://github.com/AlexEMG/DeepLabCut.git cloned-DLC-repo
%cd cloned-DLC-repo
!ls

In [0]:
%cd /content/cloned-DLC-repo/examples/openfield-Pranav-2018-10-30
!ls

In [0]:
#(this will take a few minutes to install all the dependences!)

!pip install deeplabcut

## PLEASE, click "restart runtime" from the output above before proceeding! 

In [0]:
#GUIs don't work on the cloud, so we will supress wxPython: 
import os
os.environ["DLClight"]="True"
os.environ["Colab"]="True"

import deeplabcut

In [0]:
#create a path variable that links to the config file:
path_config_file = '/content/cloned-DLC-repo/examples/openfield-Pranav-2018-10-30/config.yaml'

# Loading example data set:
deeplabcut.load_demo_data(path_config_file)

## Start training:
This function trains the network for a specific shuffle of the training dataset. 

In [0]:
#let's also change the display and save_iters just in case Colab takes away the GPU... 
#if that happens, you can reload from a saved point. Typically, you want to train to 200,000 + iterations.
#more info and there are more things you can set: https://github.com/AlexEMG/DeepLabCut/blob/master/docs/functionDetails.md#g-train-the-network

deeplabcut.train_network(path_config_file, shuffle=1, displayiters=10,saveiters=100)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 1.03M iterations). 
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....

We recommend you run this for ~1,000 iterations, just as a demo. This should take around 20 min. Note, that **when you hit "STOP" you will get a KeyInterrupt "error"! No worries! :)**

## Start evaluating:
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [0]:
%matplotlib notebook
deeplabcut.evaluate_network(path_config_file,plotting=True)

# Here you want to see a low pixel error! Of course, it can only be as good as the labeler, so be sure your labels are good!

**Check the images**:
You can go look in the newly created "evalutaion-results" folder at the images. At around 3500 iterations, the error is ~3 pixels (but this can vary on how your demo data was split for training)

## Start Analyzing videos: 
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

**On the demo data, this should take around ~ 3 min! (The demo frames are 640x480, which should run around 35 FPS on the google-provided GPU)**

In [0]:
videofile_path = ['/content/cloned-DLC-repo/examples/openfield-Pranav-2018-10-30/videos/m3v1mp4.mp4'] #Enter the list of videos to analyze.
deeplabcut.analyze_videos(path_config_file,videofile_path, videotype='.mp4')

## Create labeled video:
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. This should run around 215 FPS on the demo video!

In [0]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

## Plot the trajectories of the analyzed videos:
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [0]:
deeplabcut.plot_trajectories(path_config_file,videofile_path)